In [1]:
import moviepy.editor as mpy

In [2]:
import pydub as pdb

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [4]:
# CONSTANTS
PREVIEW_SIZE = (400, 300)
PRODUCTION_SIZE = (1024, 768)

In [5]:
import moviepy.video.fx.all as vfx
import os

In [6]:
def resize_and_fit(origin_clip, video_size):
    new_size = (0, 0) # init the value
    if origin_clip.h > origin_clip.w:
        new_size = ((origin_clip.w * video_size[1] / origin_clip.h),video_size[1])
    else:
        new_size = ((video_size[0], origin_clip.h * video_size[0] / origin_clip.w))
    clip = origin_clip.resize(new_size).on_color(video_size)
    return clip

In [7]:
def set_scene_dur(origin_clip, dur, fps=25):
    clip = origin_clip.set_duration(dur).set_fps(fps)
    return clip

In [33]:
def set_img_dur(origin_clip, dur, fps=25):
    clip = origin_clip.set_duration(dur).set_fps(fps)
    return clip

In [34]:
def set_video_dur(origin_clip, dur, fps=25):
    clip = vfx.speedx(origin_clip, final_duration=dur).set_fps(fps)
    return clip

In [35]:
def fixed_time_scene_transition(resources):
    clips = []
    for res in resources:
        # EH: use a better way to detect the type of a file
        file_type = res[-3:]
        if file_type in ["mov", "mp4", "avi", "flv"]:
            clips.append(set_video_dur(resize_and_fit(mpy.VideoFileClip(res), PREVIEW_SIZE), 2))
        elif file_type in ["jpg", "png"]:
            clips.append(set_img_dur(resize_and_fit(mpy.ImageClip(res), PREVIEW_SIZE), 2))
            
    return mpy.concatenate_videoclips(clips)

In [36]:
test_resources = sorted(os.listdir("TestResources/"))

In [37]:
combined_clip = fixed_time_scene_transition([os.path.join("TestResources", res) for res in test_resources])

In [38]:
combined_clip.ipython_display(maxduration=120)

100%|█████████▉| 650/651 [00:08<00:00, 80.54it/s] 
